In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

!pip install transformers
!pip install flask
!pip install flask-ngrok

In [ ]:
from IPython import get_ipython
import flask
import transformers
import torch
import requests
import json
import pandas as pd

from google.colab import drive

drive.mount('/content/drive')

import os
os.chdir('drive/MyDrive/hack4ukraine')

In [80]:
class TwitterClassifier:

    def __init__(self):

        self.headers = {
            'Authorization': r'Bearer AAAAAAAAAAAAAAAAAAAAABDxaAEAAAAAx2PtQqiAPWCdp%2BpRC0USF%2FEmZvs%3DdcXc6zV2m0Ixb7Cp1GVemtxMkqFIJUw8rdPh2qM8xSH7nNLQSb'
        }
        self.url = 'https://api.twitter.com/2/tweets/'

        self.tokenizer = torch.load('preprocessing.pt')
        self.model = torch.load('model.pt')
        self.model.to('cuda:0')

    
    def predict_proba(self, tweet_id: str) -> float:
        url = self.url + tweet_id
        response = requests.request('GET', url, headers=self.headers)
        content = json.loads(response.content)
        text = content['data']['text']

        tokenized = self.tokenizer(text, return_tensors='pt')
        tokenized['input_ids'] = tokenized['input_ids'].to('cuda:0')
        tokenized['token_type_ids'] = tokenized['token_type_ids'].to('cuda:0')
        tokenized['attention_mask'] = tokenized['attention_mask'].to('cuda:0')

        for item in tokenized.items():
          item[1].to('cuda:0')
        labels = torch.Tensor([1]).unsqueeze(0).long().to('cuda:0')

        with torch.no_grad():
            pred = self.model(**tokenized, labels=labels)[1]
            
        return torch.nn.functional.softmax(pred[0])[1].item()

In [81]:
classifier = TwitterClassifier()

In [85]:
get_ipython().system_raw('python app.py &')
get_ipython().system_raw('./ngrok authtoken 26cgfBNaX1pyZXPtolJmaEGBBGH_3U2HNALYekZrGMaWV9Wdy ')
get_ipython().system_raw('./ngrok http 5000 &')

In [86]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://1b6e-34-136-46-201.ngrok.io


In [ ]:
from flask import Flask, request
app = Flask(__name__)


@app.route('/')
def index():
    id = str(request.args.get('id'))
    return {'prob': classifier.predict_proba(id)}

if __name__ == '__main__':
    app.run()